In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("raw_data.csv")

## Removendo Duplicados

In [2]:
df = df.drop_duplicates(keep="first", subset=[coluna for coluna in df.columns if coluna!="crawled_at"]).reset_index(drop=True)

## Removendo Anúncios

In [3]:
filtro_de_anuncios = [_id.isnumeric() for _id in df["id"]]

In [4]:
df = df[filtro_de_anuncios].reset_index(drop=True)

## Removendo strings de campos numericos

In [5]:
set(df["rooms"])

{'-- Quarto', '1 Quarto', '2 Quartos', '3 Quartos', '4 Quartos', '5 Quartos'}

In [6]:
set(df["rooms"].str.split(" ").str[0].str.replace("--","0"))

{'0', '1', '2', '3', '4', '5'}

In [7]:
df["rooms_limpo"] = (df["rooms"]
                     .str.split(" ")
                     .str[0]
                     .str.replace("--","0")
                     .astype(int))

In [8]:
set(df["bathrooms"])

{'-- Banheiro',
 '1 Banheiro',
 '2 Banheiros',
 '3 Banheiros',
 '4 Banheiros',
 '5 Banheiros',
 '6 Banheiros',
 '7 Banheiros',
 '8 Banheiros'}

In [9]:
df["bathrooms_limpo"] = (df["bathrooms"]
                         .str.split(" ")
                         .str[0]
                         .str.replace("--","0")
                         .astype(int))

In [10]:
set(df["garages"])

{'-- Vaga',
 '1 Vaga',
 '10 Vagas',
 '14 Vagas',
 '2 Vagas',
 '3 Vagas',
 '4 Vagas',
 '47 Vagas',
 '5 Vagas',
 '6 Vagas',
 '7 Vagas',
 '8 Vagas',
 '9 Vagas'}

In [11]:
df["garages_limpo"] = (df["garages"]
                       .str.split(" ")
                       .str[0]
                       .str.replace("--","0")
                       .astype(int))

In [12]:
df["price_limpo"] = [int(w.split("R$ ")[1].replace(".","")) for w in df["price"]]

In [13]:
df["condo"] = df["condo"].fillna("MISSING")

In [14]:
df["condo_limpo"] = [int(w.split("R$ ")[1].replace(".","")) if w!="MISSING" else np.nan for w in df["condo"]]

In [15]:
df["area_limpo"] = df["area"].astype(int)

In [16]:
df["crawled_at"] = pd.to_datetime(df["crawled_at"], format="%Y-%m-%d %H:%M")

In [17]:
df = df.drop(columns=["area", "rooms", "bathrooms", "garages", "price", "condo"])

## Bairro

In [18]:
df["address"]

0         Avenida Portugal, 401 - Brooklin, São Paulo - SP
1            Rua Nova York, 470 - Brooklin, São Paulo - SP
2         Avenida Portugal, 371 - Brooklin, São Paulo - SP
3                  Rua Michigan - Brooklin, São Paulo - SP
4                                 Brooklin, São Paulo - SP
                               ...                        
15519                 Rua Purus - Tucuruvi, São Paulo - SP
15520      Rua Manuel Gaya, 945 - Tucuruvi, São Paulo - SP
15521    Avenida Coronel Sezefredo Fagundes - Tucuruvi,...
15522    Avenida Coronel Sezefredo Fagundes - Tucuruvi,...
15523       Avenida Mazzei, 530 - Tucuruvi, São Paulo - SP
Name: address, Length: 15524, dtype: object

In [19]:
df["bairro"] = df["address"].str.split("- ").str[1].str.split(",").str[0]

In [20]:
df.loc[df["bairro"].isin(["SP","RJ"]), "bairro"] = df.loc[df["bairro"].isin(["SP","RJ"]), "address"].str.split(",").str[0]

In [21]:
df["bairro"] = df["bairro"].str.lower().str.normalize("NFKD").str.encode("ascii", errors = "ignore").str.decode("utf-8").str.replace(" ", "_")

In [22]:
df["crawler"] = df["crawler"].str.lower().str.replace(" ","_")

In [23]:
df.loc[df["bairro"] != df["crawler"] , ["bairro", "crawler"]]

,bairro,crawler
3596,freguesia_do_o,fregesia_do_o
3597,freguesia_do_o,fregesia_do_o
3598,freguesia_do_o,fregesia_do_o
3599,freguesia_do_o,fregesia_do_o
3600,freguesia_do_o,fregesia_do_o
...,...,...
4521,freguesia_do_o,fregesia_do_o
4522,freguesia_do_o,fregesia_do_o
4523,freguesia_do_o,fregesia_do_o
4524,freguesia_do_o,fregesia_do_o


In [24]:
df["crawler"] = df["crawler"].str.replace("fregesia","freguesia")

In [25]:
df.loc[df["bairro"] != df["crawler"] , ["bairro", "crawler"]]

,bairro,crawler


In [26]:
df = df.drop(columns = "bairro")

## Amenities

In [27]:
df = pd.concat([df, df["amenities"].str.get_dummies("\n")], axis = 1)

## Criando novo arquivo com dados limpos

In [28]:
df.to_csv("clean_data.csv", index = False)